In the data package , the directory language_identification contains data for 5 languages. Based on this data, train an SVM classifier for language recognition between these 5 languages.
* kun regressioalgoritmi tekee luokittelun, jokaista luokkaa kohden tehdään oma luokittelija ("Onko englantia? Kyllä/ei") --> viisi decision boundaryä

In [5]:
# Reading the data in makes sense to structure a little bit
# ratkaise, miten kansi tuodaan omasta GitHubista!
# toimii myös luomalla Colabiin kansion (tässä nimeltä "texts"), 
# jonne tiedostot raahaa (kansio katoaa, kun ajo päättyy)

import random

def read_data_one_lang(lang,part):
    """Reads one file for one language. Returns data in the form of pairs of (lang,line)"""
    filename="texts/{}_{}.txt".format(lang,part)
    result=[] #this will be the list of pairs (lang,line)
    with open(filename) as f:
        for line in f:
            line=line.strip()
            result.append((lang,line)) 
    return result


def read_data_all_langs(part):
    """Reads train, test or dev data for all languages. part can be train, test, or devel"""
    data=[]
    for lang in ("en","es","et","fi","pt"):
        pairs=read_data_one_lang(lang,part)
        data.extend(pairs) #just add these lines to the end
    #...done
    #but now they come in the order of languages
    #we really must scramble these!
    random.shuffle(data)
    
    #let's yet separate the labels and lines, we will need that anyway
    labels=[label for label,line in data]
    lines=[line for label,line in data]
    return labels,lines

labels_train,lines_train=read_data_all_langs("train")
labels_dev,lines_dev=read_data_all_langs("devel")
for label,line in zip(labels_train[:5],lines_train[:5]):
    print(label,"   ",line[:30],"...")
#and beyond this point, exactly same code is applicable as before

fi     Pakko laittaa tämä toinenkin a ...
et     Milovan Djilas kinnitab oma ra ...
es     Con Pablo Mackenna como invita ...
fi     Hän kuitenkin toivoi, että rau ...
es     Este grupo priorizaba el reagr ...


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.svm

vectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
feature_matrix_train=vectorizer.fit_transform(lines_train)
feature_matrix_dev=vectorizer.transform(lines_dev)

for C in (0.001,0.01,0.1,1,10,100):
    classifier=sklearn.svm.LinearSVC(C=C)
    classifier.fit(feature_matrix_train, labels_train)
    print("C=",C,"     ",classifier.score(feature_matrix_dev, labels_dev))

C= 0.001       0.8758
C= 0.01       0.9144
C= 0.1       0.933
C= 1       0.9302
C= 10       0.9102
C= 100       0.8726


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


* 93% is now that great!
* Are words actually a good source of features?
* Let us try with character n-grams instead of words

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.svm

vectorizer=CountVectorizer(max_features=100000,binary=True,
                           ngram_range=(1,3),analyzer="char_wb")
feature_matrix_train=vectorizer.fit_transform(lines_train)
feature_matrix_dev=vectorizer.transform(lines_dev)

for C in (0.001,0.01,0.1,1,10,100):
    classifier=sklearn.svm.LinearSVC(C=C)
    classifier.fit(feature_matrix_train, labels_train)
    print("C=",C,"     ",classifier.score(feature_matrix_dev, labels_dev))


C= 0.001       0.9762
C= 0.01       0.9778
C= 0.1       0.9732
C= 1       0.9726
C= 10       0.9726
C= 100       0.9724


Now, that's quite a bit better!